In [1]:
#i3d extraction

!git clone https://github.com/piergiaj/pytorch-i3d.git
%cd pytorch-i3d
!pip install torch torchvision numpy pillow tqdm

# Create models directory
!mkdir -p models

# Download flow_imagenet.pt into models/
!wget -O /kaggle/working/flow_imagenet.pt https://www.dropbox.com/s/7w4z5q9fowcp9x5/flow_imagenet.pt?dl=1


fatal: destination path 'pytorch-i3d' already exists and is not an empty directory.
/kaggle/working/pytorch-i3d
--2025-04-16 06:26:52--  https://www.dropbox.com/s/7w4z5q9fowcp9x5/flow_imagenet.pt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.18, 2620:100:601f:18::a27d:912
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘/kaggle/working/flow_imagenet.pt’

/kaggle/working/flo     [ <=>                ]  70.00K  --.-KB/s    in 0.06s   

2025-04-16 06:26:53 (1.16 MB/s) - ‘/kaggle/working/flow_imagenet.pt’ saved [71685]



In [2]:
!ls /kaggle/working/pytorch-i3d

models	pytorch-i3d


In [3]:
import sys
sys.path.append('/kaggle/working/pytorch-i3d')
from pytorch_i3d import InceptionI3d



!wget -O /kaggle/working/flow_imagenet.pt https://github.com/piergiaj/pytorch-i3d/raw/master/models/flow_imagenet.pt
!file /kaggle/working/flow_imagenet.pt
!wget -O /kaggle/working/rgb_imagenet.pt https://github.com/piergiaj/pytorch-i3d/raw/master/models/rgb_imagenet.pt


ModuleNotFoundError: No module named 'pytorch_i3d'

# Final I3D Extraction 


In [ ]:
import os
import numpy as np
import torch
from PIL import Image
from tqdm import tqdm
import torchvision.transforms as transforms
from pytorch_i3d import InceptionI3d

# --- Config ---
dataset_path = '/kaggle/input/shanghaitech-anomaly-detection/dataset/mp'
output_dir = '/kaggle/working/I3D_Train_Frame'
os.makedirs(output_dir, exist_ok=True)

# --- Device Setup ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- I3D Model Loader (RGB version) ---
def load_i3d_model():
    i3d = InceptionI3d(400, in_channels=3)  # 3 channels for RGB
    i3d.load_state_dict(torch.load('/kaggle/working/rgb_imagenet.pt'))
    i3d.to(device)
    i3d.eval()
    return i3d

# --- I3D Feature Extractor (1024×7×7) ---
class I3DFeatureExtractor(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.features = torch.nn.Sequential(
            model.Conv3d_1a_7x7,
            model.MaxPool3d_2a_3x3,
            model.Conv3d_2b_1x1,
            model.Conv3d_2c_3x3,
            model.MaxPool3d_3a_3x3,
            model.Mixed_3b,
            model.Mixed_3c,
            model.MaxPool3d_4a_3x3,
            model.Mixed_4b,
            model.Mixed_4c,
            model.Mixed_4d,
            model.Mixed_4e,
            model.Mixed_4f,
            model.MaxPool3d_5a_2x2,
            model.Mixed_5b,
            model.Mixed_5c
        )

    def forward(self, x):
        x = x.unsqueeze(2)  # (N, C, 1, H, W)
        x = self.features(x)
        # Get rid of the batch dimension and time dimension but keep 1024×7×7
        return x.squeeze(0).squeeze(0)  # Remove batch and time dimensions, keep C×H×W
        print(x.shape)

# --- RGB Frame Preprocessing ---
def preprocess_rgb_frame(frame_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
    ])
    img = Image.open(frame_path).convert('RGB')
    return transform(img).to(device)

# --- Feature Extraction ---
def extract_i3d_features(model, dataset_path, output_dir):
    extractor = I3DFeatureExtractor(model)
   
    for video_folder in tqdm(sorted(os.listdir(dataset_path))):
        # Change from 'optical_flow' to 'frames' directory for RGB frames
        frames_dir = os.path.join(dataset_path, video_folder, 'frames')
        if not os.path.exists(frames_dir):
            continue
           
        frame_paths = sorted([os.path.join(frames_dir, f)
                          for f in os.listdir(frames_dir)
                          if f.lower().endswith(('.jpg','.png','.jpeg'))])
       
        # Create output directory for this video's RGB features
        video_output_dir = os.path.join(output_dir, video_folder)
        os.makedirs(video_output_dir, exist_ok=True)
       
        for frame_idx, frame_path in enumerate(frame_paths):
            try:
                rgb_frame = preprocess_rgb_frame(frame_path)
                with torch.no_grad():
                    # This will return a tensor of shape (1024, 7, 7)
                    features = extractor(rgb_frame.unsqueeze(0)).cpu().numpy()
               
                # Save the 1024×7×7 feature tensor directly
                np.save(os.path.join(video_output_dir, f"rgb_frame_{frame_idx:04d}.npy"), features)
            except Exception as e:
                print(f"Error processing {frame_path}: {e}")
                continue

if __name__ == "__main__":
    print("Loading I3D RGB model...")
    i3d = load_i3d_model()
   
    print("Extracting 1024×7×7 RGB frame features...")
    extract_i3d_features(i3d, dataset_path, output_dir)
   
    # Verify feature dimensions
    sample_video = os.listdir(output_dir)[0]
    sample_frame = os.listdir(os.path.join(output_dir, sample_video))[0]
    sample_features = np.load(os.path.join(output_dir, sample_video, sample_frame))
    print(f"\nFeature verification:")
    print(f"Shape: {sample_features.shape}")  # Should be (1024, 7, 7)
    print(f"Value range: [{sample_features.min():.3f}, {sample_features.max():.3f}]")
   
    print(f"\n✅ RGB frame features with 1024×7×7 dimensions saved in {output_dir}")

# Flow I3D Extraction

In [ ]:
import os
import numpy as np
import torch
from PIL import Image
from tqdm import tqdm
import torchvision.transforms as transforms
from pytorch_i3d import InceptionI3d

# --- Config ---
dataset_path = '/kaggle/input/shanghaitech-anomaly-detection/dataset/mp'
output_dir = '/kaggle/working/I3D_Train_Flow'
os.makedirs(output_dir, exist_ok=True)

# --- Device Setup ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- I3D Model Loader (Optical Flow version) ---
def load_i3d_model():
    i3d = InceptionI3d(400, in_channels=2)  # 2 channels for optical flow
    i3d.load_state_dict(torch.load('/kaggle/working/flow_imagenet.pt'))
    i3d.to(device)
    i3d.eval()
    return i3d

# --- I3D Feature Extractor (1024-D) ---
class I3DFeatureExtractor(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.features = torch.nn.Sequential(
            model.Conv3d_1a_7x7,
            model.MaxPool3d_2a_3x3,
            model.Conv3d_2b_1x1,
            model.Conv3d_2c_3x3,
            model.MaxPool3d_3a_3x3,
            model.Mixed_3b,
            model.Mixed_3c,
            model.MaxPool3d_4a_3x3,
            model.Mixed_4b,
            model.Mixed_4c,
            model.Mixed_4d,
            model.Mixed_4e,
            model.Mixed_4f,
            model.MaxPool3d_5a_2x2,
            model.Mixed_5b,
            model.Mixed_5c
        )

    def forward(self, x):
        x = x.unsqueeze(2)  # (N, C, 1, H, W)
        x = self.features(x)
        print(x.shape)
        return x

# --- Optical Flow Preprocessing ---
def preprocess_flow_frame(flow_frame_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x[:2]),  # Take only first two channels
        transforms.Normalize(mean=[0.5, 0.5], std=[0.5, 0.5])  # Normalize to [-1,1]
    ])
    img = Image.open(flow_frame_path)
    return transform(img).to(device)

# --- Feature Extraction ---
def extract_i3d_features(model, dataset_path, output_dir):
    extractor = I3DFeatureExtractor(model)
   
    for video_folder in tqdm(sorted(os.listdir(dataset_path))):
        flow_dir = os.path.join(dataset_path, video_folder, 'optical_flow')
        if not os.path.exists(flow_dir):
            continue
           
        flow_paths = sorted([os.path.join(flow_dir, f)
                          for f in os.listdir(flow_dir)
                          if f.lower().endswith(('.jpg','.png','.jpeg'))])
       
        # Create output directory for this video's flow features
        video_output_dir = os.path.join(output_dir, video_folder)
        os.makedirs(video_output_dir, exist_ok=True)
       
        for frame_idx, flow_path in enumerate(flow_paths):
            try:
                flow_frame = preprocess_flow_frame(flow_path)
                with torch.no_grad():
                    features = extractor(flow_frame.unsqueeze(0)).cpu().numpy()  # (1,1024)
         
               
                # Save each frame's flow features as separate numpy file
                frame_features = features.squeeze()  # shape (1024,)
                np.save(os.path.join(video_output_dir, f"flow_frame_{frame_idx:04d}.npy"), frame_features)
                #arr=np.load(os.path.join(video_output_dir, f"flow_frame_{frame_idx:04d}.npy"))
                #print(arr.shape)
            except Exception as e:
                print(f"Error processing {flow_path}: {e}")
                continue

if __name__ == "__main__":
    print("Loading I3D flow model...")
    i3d = load_i3d_model()
   
    print("Extracting 1024-D flow features...")
    extract_i3d_features(i3d, dataset_path, output_dir)
   
    # Verify feature dimensions
    sample_video = os.listdir(output_dir)[0]
    sample_frame = os.listdir(os.path.join(output_dir, sample_video))[0]
    sample_features = np.load(os.path.join(output_dir, sample_video, sample_frame))
    print(f"\nFeature verification:")
    print(f"Shape: {sample_features.shape}")  # Should be (1024,7,7)
    print(f"First 5 dimensions: {sample_features[:5]}")
    print(f"Value range: [{sample_features.min():.3f}, {sample_features.max():.3f}]")
   
    print(f"\n✅ Optical flow features saved as individual files in {output_dir}")

# Concadnation

In [ ]:
import os
import numpy as np
import glob
from tqdm import tqdm

# --- Config ---
rgb_features_dir = '/kaggle/working/I3D_Train_Frame'
flow_features_dir = '/kaggle/working/I3D_Test_Flow'
output_dir = '/kaggle/working/I3D_Combined_Features'
os.makedirs(output_dir, exist_ok=True)

def concatenate_rgb_flow_features():
    """
    Concatenate RGB features (1024×7×7) and flow features (1024×7×7)
    from two different directories into combined features (2048×7×7).
    """
    print("Concatenating RGB and flow features to create 2048×7×7 feature tensors...")
    
    # Get all video folders from RGB directory
    rgb_video_folders = set(os.listdir(rgb_features_dir))
    flow_video_folders = set(os.listdir(flow_features_dir))
    
    # Find common videos in both directories
    common_videos = rgb_video_folders.intersection(flow_video_folders)
    print(f"Found {len(common_videos)} videos with both RGB and flow features")
    
    for video_folder in tqdm(sorted(common_videos)):
        rgb_video_path = os.path.join(rgb_features_dir, video_folder)
        flow_video_path = os.path.join(flow_features_dir, video_folder)
        
        # Get all RGB feature files
        rgb_feature_files = sorted(glob.glob(os.path.join(rgb_video_path, "*.npy")))
        
        # Get all flow feature files
        flow_feature_files = sorted(glob.glob(os.path.join(flow_video_path, "*.npy")))
        
        if not rgb_feature_files or not flow_feature_files:
            print(f"Missing features for {video_folder}")
            continue
        
        # Create output directory for this video
        video_output_dir = os.path.join(output_dir, video_folder)
        os.makedirs(video_output_dir, exist_ok=True)
        
        # Process each frame that has both RGB and flow features
        for i in range(min(len(rgb_feature_files), len(flow_feature_files))):
            rgb_file = rgb_feature_files[i]
            flow_file = flow_feature_files[i]
            
            # Extract frame number from filename
            rgb_frame_num = os.path.basename(rgb_file).split('.')[0].split('_')[-1]
            flow_frame_num = os.path.basename(flow_file).split('.')[0].split('_')[-1]
            
            try:
                # Load RGB and flow features
                rgb_feat = np.load(rgb_file)
                flow_feat = np.load(flow_file)
                
                # Check shapes
                if rgb_feat.shape != (1024, 7, 7) or flow_feat.shape != (1024, 7, 7):
                    print(f"Unexpected shape: RGB {rgb_feat.shape}, Flow {flow_feat.shape}")
                    continue
                
                # Concatenate along the channel dimension (first dimension)
                combined = np.concatenate([rgb_feat, flow_feat], axis=0)
                
                # Save combined feature
                output_path = os.path.join(video_output_dir, f"combined_frame_{rgb_frame_num}.npy")
                np.save(output_path, combined)
            except Exception as e:
                print(f"Error processing frame {rgb_frame_num} of {video_folder}: {e}")
                continue

if __name__ == "__main__":
    concatenate_rgb_flow_features()
    
    # Verify concatenated features
    try:
        sample_video = os.listdir(output_dir)[0]
        sample_frame = os.listdir(os.path.join(output_dir, sample_video))[0]
        sample_path = os.path.join(output_dir, sample_video, sample_frame)
        sample_features = np.load(sample_path)
        
        print(f"\nFeature verification:")
        print(f"File: {sample_path}")
        print(f"Shape: {sample_features.shape}")  # Should be (2048, 7, 7)
        print(f"Value range: [{sample_features.min():.3f}, {sample_features.max():.3f}]")
    except (IndexError, FileNotFoundError) as e:
        print(f"Could not verify features: {e}")
    
    print(f"\n✅ Combined RGB+Flow features with 2048×7×7 shape saved in {output_dir}")

In [5]:
import shutil
import os

working_dir = '/kaggle/working/'

for item in os.listdir(working_dir):
    item_path = os.path.join(working_dir, item)
    try:
        if os.path.isdir(item_path):
            shutil.rmtree(item_path)
        else:
            os.remove(item_path)
        print(f"🗑️ Deleted: {item_path}")
    except Exception as e:
        print(f"⚠️ Could not delete {item_path}: {e}")



🗑️ Deleted: /kaggle/working/flow_imagenet.pt
🗑️ Deleted: /kaggle/working/.zip
🗑️ Deleted: /kaggle/working/.virtual_documents
🗑️ Deleted: /kaggle/working/rgb_imagenet.pt
🗑️ Deleted: /kaggle/working/state.db
🗑️ Deleted: /kaggle/working/pytorch-i3d
🗑️ Deleted: /kaggle/working/I3D_Features_flowframetrain
🗑️ Deleted: /kaggle/working/combined_rgb_flow_features
